# Sequential Orchestration Pattern with Semantic Kernel

This notebook demonstrates the Sequential Orchestration pattern using Microsoft Semantic Kernel in Python. Sequential orchestration involves executing tasks in a predefined order, where the output of one step is used as input for the next step. This approach is ideal for workflows that require strict dependencies between steps.

### Key Features:
- **Step-by-Step Execution**: Tasks are executed sequentially, ensuring proper order and dependencies.
- **Task Specialization**: Each step is designed to perform a specific function or skill.
- **Error Handling**: Mechanisms are implemented to manage errors at each step.

### Examples:
1. **Recipe Creation**: Extract ingredients, create step-by-step instructions, and review the recipe for clarity and completeness.
2. **Document Processing**: Summarize a document, translate it, and ensure the translation is accurate and readable.

The notebook utilizes `SequentialPlanner` and Semantic Kernel's agent orchestration framework to manage and execute tasks sequentially, showcasing the power and flexibility of Semantic Kernel.

In [ ]:
# Example: Sequential Orchestration for Recipe Creation
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents.orchestration.sequential import SequentialOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
import asyncio

# Define agents
ingredient_extractor_agent = ChatCompletionAgent(
    name="IngredientExtractorAgent",
    instructions=(
        "You are a culinary expert. Given a dish name, list all the ingredients required to prepare it."
    ),
    service=AzureChatCompletion(
          api_key="**",
          endpoint="***",
          deployment_name= "****"
    ),
)

step_creator_agent = ChatCompletionAgent(
    name="StepCreatorAgent",
    instructions=(
        "You are a recipe writer. Given a list of ingredients, create step-by-step instructions to prepare the dish."
    ),
    service=AzureChatCompletion(
          api_key="**",
          endpoint="***",
          deployment_name= "****"
    ),
)

review_agent = ChatCompletionAgent(
    name="ReviewAgent",
    instructions=(
        "You are a food critic. Given the recipe steps, review them for clarity and completeness, and suggest improvements if necessary."
    ),
    service=AzureChatCompletion(
          api_key="**",
          endpoint="***",
          deployment_name= "*****"
    ),
)

# Set up sequential orchestration
agents = [ingredient_extractor_agent, step_creator_agent, review_agent]
sequential_orchestration = SequentialOrchestration(members=agents)

# Start runtime
runtime = InProcessRuntime()
runtime.start()

# Invoke orchestration
try:
    print("Starting orchestration...")
    orchestration_result = await sequential_orchestration.invoke(
        task="Extract ingredients for spaghetti carbonara, create step-by-step instructions using those ingredients, and review the instructions for clarity and completeness.",
        runtime=runtime,
    )

    # Collect results
    print("Waiting for orchestration result...")
    value = await orchestration_result.get(timeout=120)  # Increased timeout to 120 seconds
    print("***** Final Result *****")
    for i, step_output in enumerate(value, start=1):
        agent_name = agents[i - 1].name if i - 1 < len(agents) else "Unknown Agent"
        try:
            if isinstance(step_output, tuple):
                # Handle ChatCompletion objects
                if hasattr(step_output[1], 'choices') and step_output[1].choices:
                    content = step_output[1].choices[0].message.content
                    print(f"{agent_name}: {content}")
                else:
                    print(f"{agent_name}: No meaningful content available")
            else:
                print(f"{agent_name}: {step_output}")
        except Exception as e:
            print(f"Error processing output for {agent_name}: {e}")
except asyncio.TimeoutError:
    print("The orchestration timed out. Consider increasing the timeout or debugging agent execution.")
except Exception as e:
    import traceback
    print("An error occurred during orchestration:")
    print(traceback.format_exc())

# Stop runtime
await runtime.stop_when_idle()

Starting orchestration...
Waiting for orchestration result...
***** Final Result *****
IngredientExtractorAgent: ### Recipe Review:

The recipe you've laid out for classic guacamole is clear, relatively thorough, and straightforward, with good attention to detail and organization. However, there’s room to improve clarity in a few places, avoid potential confusion, and cater to varying skill levels. Below, I'll provide a review of the recipe's strengths followed by areas for improvement and recommendations.

---

### Strengths:
1. **Ingredient List**: 
   - The inclusion of optional ingredients for flavor variation (such as the garlic and jalapeño) is thoughtful and provides flexibility.
   - Measurements are clear, and the use of terms like “to taste” for subjective elements like salt is appropriate.

2. **Organization**: 
   - Ingredients are prepped in advance (Step 1), which is helpful for beginners and keeps the workflow tidy.
   - The recipe follows a logical progression from prep